In [1]:
import numpy as np
import tensorflow as tf
import tensorflow.keras.layers as layers
from tensorflow.keras.models import save_model

In [2]:
# Cifar10データロード
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

In [3]:
# 正規化
x_train = x_train/255.0
x_test = x_test/255.0

In [4]:
# CNN構築
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(
        24, 3, activation='relu', padding='same',
        input_shape=(32, 32, 3)),  # (32, 32, 24)
    tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'),  # (30, 30, 32)
    tf.keras.layers.MaxPooling2D(pool_size=2, strides=2, padding='valid'), # (15, 15, 32)
    tf.keras.layers.Dropout(0.25),

    tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu', padding='same'),  # (15, 15, 64)
    tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'),  # (13, 13, 64)
    tf.keras.layers.MaxPooling2D(pool_size=2, strides=2, padding='valid'), # (6, 6, 64)
    tf.keras.layers.Dropout(0.25),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Activation('softmax')
])

In [5]:
# モデル可視化
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 24)        672       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 30, 30, 32)        6944      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 15, 15, 64)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 13, 13, 32)        18464     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 32)          0

In [6]:
# モデルチェックポイントのコールバック
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    '02_cnn_cifar10.hdf5', verbose=1, save_weights_only=False)

In [7]:
# モデルコンパイル
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [8]:
# 訓練
model.fit(
    x_train,
    y_train,
    epochs=15,
    validation_data=(x_test, y_test),
    callbacks=[cp_callback]
)

Train on 50000 samples, validate on 10000 samples
Epoch 1/15
49760/50000 [============================>.] - ETA: 0s - loss: 1.9413 - accuracy: 0.2957
Epoch 00001: saving model to 02_cnn_cifar10.hdf5
50000/50000 [==============================] - 16s 310us/sample - loss: 1.9393 - accuracy: 0.2965 - val_loss: 1.4235 - val_accuracy: 0.5214
Epoch 2/15
49792/50000 [============================>.] - ETA: 0s - loss: 1.3870 - accuracy: 0.5367
Epoch 00002: saving model to 02_cnn_cifar10.hdf5
50000/50000 [==============================] - 13s 252us/sample - loss: 1.3863 - accuracy: 0.5370 - val_loss: 1.2867 - val_accuracy: 0.5779
Epoch 3/15
49792/50000 [============================>.] - ETA: 0s - loss: 1.2406 - accuracy: 0.5924
Epoch 00003: saving model to 02_cnn_cifar10.hdf5
50000/50000 [==============================] - 13s 253us/sample - loss: 1.2407 - accuracy: 0.5927 - val_loss: 1.1161 - val_accuracy: 0.6339
Epoch 4/15
49920/50000 [============================>.] - ETA: 0s - loss: 1.1462 - 

In [ ]:
# モデル評価
val_loss, val_acc = model.evaluate(x_test, y_test, batch_size=128)

In [ ]:
# 保存したモデルのロード
load_model = tf.keras.models.load_model("02_cnn_cifar10.hdf5")

In [ ]:
# テスト画像を1枚ロード
from IPython.display import Image, display_png
from tensorflow.keras.preprocessing.image import img_to_array, load_img

img = tf.keras.preprocessing.image.load_img('image/automobile10.png', False, target_size=(32, 32))
display_png(img)

In [ ]:
# 入力画像成形、および正規化
x = img_to_array(img)
x = x.reshape(-1, 32, 32, 3)
x = x.astype('float32')
x /= 255

In [ ]:
# 推論実行
predict_result = load_model.predict(x)

In [ ]:
print(predict_result)
print(np.squeeze(predict_result)) # 不要な次元を削除
print(np.argmax(np.squeeze(predict_result)))